In [1]:
import io
import torch
import torch.nn as nn
import math
from collections import OrderedDict
import torchvision.transforms as transforms
from PIL import Image
import os

In [2]:
# This code will convert the weights form the pretrained models to be able to use them for Python 3

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
################## AlexNet ##################

def bn_relu(inplanes):
    return nn.Sequential(nn.BatchNorm2d(inplanes), nn.ReLU(inplace=True))

def bn_relu_pool(inplanes, kernel_size=3, stride=2):
    return nn.Sequential(nn.BatchNorm2d(inplanes), nn.ReLU(inplace=True), nn.MaxPool2d(kernel_size=kernel_size, stride=stride))

class AlexNet(nn.Module):
    def __init__(self, num_classes=1):
        super(AlexNet, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, bias=False)
        self.relu_pool1 = bn_relu_pool(inplanes=96)
        self.conv2 = nn.Conv2d(96, 192, kernel_size=5, padding=2, groups=2, bias=False)
        self.relu_pool2 = bn_relu_pool(inplanes=192)
        self.conv3 = nn.Conv2d(192, 384, kernel_size=3, padding=1, groups=2, bias=False)
        self.relu3 = bn_relu(inplanes=384)
        self.conv4 = nn.Conv2d(384, 384, kernel_size=3, padding=1, groups=2, bias=False)
        self.relu4 = bn_relu(inplanes=384)
        self.conv5 = nn.Conv2d(384, 256, kernel_size=3, padding=1, groups=2, bias=False)
        self.relu_pool5 = bn_relu_pool(inplanes=256)
        # classifier
        self.conv6 = nn.Conv2d(256, 256, kernel_size=5, groups=2, bias=False)
        self.relu6 = bn_relu(inplanes=256)
        self.conv7 = nn.Conv2d(256, num_classes, kernel_size=1, bias=False)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu_pool1(x)
        x = self.conv2(x)
        x = self.relu_pool2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.conv5(x)
        x = self.relu_pool5(x)
        x = self.conv6(x)
        x = self.relu6(x)
        x = self.conv7(x)
        x = x.view(x.size(0), -1)
        return x

In [5]:
def byte_convert(model: dict) -> dict:
    new_model = dict()
    for key in model:
        if type(key) == bytes:
            new_key = key.decode("utf-8")
        else:
            new_key = key
        new_model[new_key] = dict()
        if isinstance(model[key], dict):
            new_model[new_key] = byte_convert(model[key])
        else:
            new_model[new_key] = model[key]
    return new_model

In [6]:
def load_model(model_path: str) -> dict:
    return torch.load(model_path, encoding='bytes', map_location=device)

In [7]:
def load_model2(model_path: str) -> nn.Module:
    with open(model_path, 'rb') as f:
        buffer = io.BytesIO(f.read())
        return torch.load(buffer, map_location=device)

In [17]:
def save_model(model: nn.Module, state_dict: dict) -> None:
    model.load_state_dict(state_dict)
    model.eval()
    print(model.eval())
    torch.save(model, type(model).__name__ + '.pth')

In [18]:
model = AlexNet()
model = load_model('pytorch-models/alexnet.pth')

In [19]:
model.keys()

dict_keys([b'optimizer', b'epoch', b'state_dict', b'best_prec1'])

In [20]:
state_dict = byte_convert(model)['state_dict']

In [21]:
save_model(AlexNet(), state_dict)

AlexNet(
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4), bias=False)
  (relu_pool1): Sequential(
    (0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Conv2d(96, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=2, bias=False)
  (relu_pool2): Sequential(
    (0): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2, bias=False)
  (relu3): Sequential(
    (0): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): ReLU(inplace=True)
  )
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2, bias=False)
  (

In [16]:
model.eval()

AttributeError: 'dict' object has no attribute 'eval'

In [13]:
def read_img(root, filedir, transform=None):
    # Data loading
    with open(filedir, 'r') as f:
        lines = f.readlines()  
    output = []    
    for line in lines:
        linesplit = line.split('\n')[0].split(' ')
        #print(linesplit)
        addr = linesplit[0]
        target = torch.Tensor([float(linesplit[1])])
        img = Image.open(os.path.join(root, addr)).convert('RGB')

        if transform is not None:
            img = transform(img)
        
        output.append([img, target])

    return output


In [14]:

def load_model(pretrained_dict, new):
    model_dict = new.state_dict()
    #print(new.state_dict())
    # 1. filter out unnecessary keys
    pretrained_dict = {k: v for k, v in pretrained_dict.state_dict().items() if k in model_dict}
    # 2. overwrite entries in the existing state dict
    model_dict.update(pretrained_dict)
    new.load_state_dict(model_dict)

In [7]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [21]:
# Scaling of data
valdir = './data/1/test16morph.txt'
mylines = []                             # Declare an empty list named mylines.
with open(valdir, 'rt') as myfile: # Open lorem.txt for reading text data.
    for myline in myfile:                # For each line, stored as myline,
        mylines.append(float(myline.split()[1]))       # add its contents to mylines.
print(mylines)   

min_val = min(mylines)
max_val = max(mylines)
lower_scale = 1
upper_scale = 5

for i in range(len(mylines)):
    mylines[i] = ((upper_scale-lower_scale)*((mylines[i]-min_val)/(max_val-min_val)))+lower_scale
    
print(mylines)


[3.395, 3.857, 3.442, 3.744, 3.238, 3.512, 3.69, 3.6682, 3.818, 3.227, 3.0, 3.238, 3.232, 3.744, 3.395]
[2.843640606767794, 5.0, 3.063010501750292, 4.4725787631271885, 2.110851808634772, 3.389731621936989, 4.22053675612602, 4.118786464410735, 4.817969661610268, 2.059509918319719, 1.0, 2.110851808634772, 2.082847141190199, 4.4725787631271885, 2.843640606767794]


In [28]:

def main():
    # net definition 
    net = AlexNet()
    # net = Nets.ResNet(block = Nets.BasicBlock, layers = [2, 2, 2, 2], num_classes = 1).cuda()

    # load pretrained model
    load_model(torch.load('AlexNet.pth', map_location=torch.device('cpu')), net) #load_model('pytorch-models/alexnet.pth')
    # load_model(torch.load('./models/resnet18.pth'), net)

    # evaluate
    net.eval()

    # loading data...
    #root = 'C:/Users/Lenovo/Documents/DTU-AP/SCUT-FBP5500_v2.1/SCUT-FBP5500_v2/Images'
    #valdir = './data/1/test1.txt'
    #root = 'C:/Users/Lenovo/Documents/DTU-AP/Multi-Morph/asian/af/asian_female_16'
    #valdir = './data/1/morph16.txt'
    #valdir = './data/1/test1.txt'
    root = 'C:/Users/Lenovo/Documents/AdvancedProject/NeuralNetwork/data/morph16'
    valdir = './data/1/test16morph.txt'
    
    print('hér')
    print(root, valdir)
    transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])  
    val_dataset = read_img(root, valdir, transform=transform)
    
    
    with torch.no_grad():
        label = []
        pred = []

        for i, (img, target) in enumerate(val_dataset):
            img = img.unsqueeze(0)#.cuda(non_blocking=True)
            target = target#.cuda(non_blocking=True)
            output = net(img).squeeze(1)
            label.append(target.cpu()[0])
            pred.append(output.cpu()[0])
            print(i)

        # measurements
        label = np.array(label)
        pred = np.array(pred)
        correlation = np.corrcoef(label, pred)[0][1]
        mae = np.mean(np.abs(label - pred))
        rmse = np.sqrt(np.mean(np.square(label - pred)))
    
    print('\n Label Array: ' + str(label))
    print('Label Mean: ' + str(label.mean()) + '\n')
    print('Prediction Array: ' + str(pred))
    print('Prediction Mean: ' + str(pred.mean()) + '\n')
    
    print('Correlation:{correlation:.4f}\t'
          'Mae:{mae:.4f}\t'
          'Rmse:{rmse:.4f}\t'.format(
            correlation=correlation, mae=mae, rmse=rmse))


if __name__ == '__main__':
    main()
    

hér
C:/Users/Lenovo/Documents/AdvancedProject/NeuralNetwork/data/morph16 ./data/1/test16morph.txt
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14

 Label Array: [3.395  3.857  3.442  3.744  3.238  3.512  3.69   3.6682 3.818  3.227
 3.     3.238  3.232  3.744  3.395 ]
Label Mean: 3.4800134

Prediction Array: [3.6961493 3.8071668 2.9782896 2.1124601 2.2001696 2.9628916 2.1310554
 2.9341388 2.0775914 2.7377534 2.9297938 2.187355  2.2512226 3.3951898
 3.5732355]
Prediction Mean: 2.7982974

Correlation:0.1036	Mae:0.7456	Rmse:0.9241	


In [ ]:
print('hér')
    print(root, valdir)
    transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])  
    val_dataset = read_img(root, valdir, transform=transform)
    
    
    with torch.no_grad():
        label = []
        pred = []

        for i, (img, target) in enumerate(val_dataset):
            img = img.unsqueeze(0)#.cuda(non_blocking=True)
            target = target#.cuda(non_blocking=True)
            output = net(img).squeeze(1)
            label.append(target.cpu()[0])
            pred.append(output.cpu()[0])
            print(i)

        # measurements
        label = np.array(label)
        pred = np.array(pred)
        correlation = np.corrcoef(label, pred)[0][1]
        mae = np.mean(np.abs(label - pred))
        rmse = np.sqrt(np.mean(np.square(label - pred)))
    
    print('\n Label Array: ' + str(label))
    print('Label Mean: ' + str(label.mean()) + '\n')
    print('Prediction Array: ' + str(pred))
    print('Prediction Mean: ' + str(pred.mean()) + '\n')
    
    print('Correlation:{correlation:.4f}\t'
          'Mae:{mae:.4f}\t'
          'Rmse:{rmse:.4f}\t'.format(
            correlation=correlation, mae=mae, rmse=rmse))


if __name__ == '__main__':
    main()

In [18]:
label

NameError: name 'label' is not defined

In [17]:
val_dataset

NameError: name 'val_dataset' is not defined